In [2]:
USE tempdb;
GO

-- recipe 25-1
IF OBJECT_ID('dbo.Book') IS NOT NULL DROP TABLE dbo.Book;
IF OBJECT_ID('dbo.INS_Book','P') IS NOT NULL DROP PROCEDURE dbo.INS_Book;
CREATE TABLE dbo.Book
        (
         BookID INT IDENTITY PRIMARY KEY,
         ISBNNBR CHAR(13) NOT NULL,
         BookNM VARCHAR(250) NOT NULL,
         AuthorID INT NOT NULL,
         ChapterDesc XML NULL
        );
GO
CREATE PROCEDURE dbo.INS_Book
       @ISBNNBR CHAR(13),
       @BookNM VARCHAR(250),
       @AuthorID INT,
       @ChapterDesc XML
AS 
INSERT  dbo.Book
        (ISBNNBR,
         BookNM,
         AuthorID,
         ChapterDesc)
VALUES  (@ISBNNBR,
         @BookNM,
         @AuthorID,
         @ChapterDesc);
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.029

In [3]:
-- recipe 25-2

INSERT  dbo.Book
        (ISBNNBR,
         BookNM,
         AuthorID,
         ChapterDesc)
VALUES  ('9781484200629',
         'SOL Server 2014 T-S0L Recipes',
         55,
'<Book name="SQL Server 2014 T-SQL Recipes">
<Chapters>
<Chapter id="1">Getting Started with SELECT</Chapter>
<Chapter id="2">Elementary Programming</Chapter>
<Chapter id="3">Working with NULLs</Chapter>
<Chapter id="4">Combining Data from Multiple Tables</Chapter>
</Chapters>
</Book>');

DECLARE @Book XML;
SET @Book =
CAST('<Book name="S0L Server 2014 Fast Answers">
<Chapters>
<Chapter id="1"> Installation, Upgrades... </Chapter>
<Chapter id="2"> Configuring SQL Server </Chapter>
<Chapter id="3"> Creating and Configuring Databases </Chapter>
<Chapter id="4"> SQL Server Agent and SQL Logs </Chapter>
</Chapters>
</Book>' as XML);

INSERT  dbo.Book
        (ISBNNBR,
         BookNM,
         AuthorID,
         ChapterDesc)
VALUES  ('1590591615',
         'SOL Server 2014 Fast Answers',
         55,
         @Book);
GO

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.013

In [4]:
-- recipe 25-3
IF EXISTS (SELECT * FROM sys.xml_schema_collections WHERE name = 'BookStoreCollection')
DROP XML SCHEMA COLLECTION BookStoreCollection;
GO
CREATE XML SCHEMA COLLECTION BookStoreCollection
AS
N'<xsd:schema  targetNamespace="http://PROD/BookStore"
               xmlns:xsd="http://www.w3.org/2001/XMLSchema"
               xmlns:sqltypes="http://schemas.microsoft.com/sqlserver/2004/sqltypes"
               elementFormDefault="qualified">
    <xsd:import  namespace="http://schemas.microsoft.com/sqlserver/2004/sqltypes"/>
    <xsd:element  name="Book">
        <xsd:complexType>
            <xsd:sequence>
                <xsd:element  name="BookName"  minOccurs="0">
                    <xsd:simpleType>
                        <xsd:restriction        base="sqltypes:varchar">
                            <xsd:maxLength  value="50"    />
                        </xsd:restriction>
                    </xsd:simpleType>
                </xsd:element>
                <xsd:element  name="ChapterID"  type="sqltypes:int" minOccurs="0"    />
                <xsd:element  name="ChapterNM"  minOccurs="0">
                    <xsd:simpleType>
                        <xsd:restriction        base="sqltypes:varchar">
                            <xsd:maxLength  value="50"    />
                        </xsd:restriction>
                    </xsd:simpleType>
                </xsd:element>
            </xsd:sequence>
        </xsd:complexType>
    </xsd:element>
</xsd:schema>';
GO

IF OBJECT_ID('dbo.BookInfoExport') IS NOT NULL DROP TABLE dbo.BookInfoExport;
CREATE TABLE dbo.BookInfoExport
       (
        BookID INT IDENTITY PRIMARY KEY,
        ISBNNBR CHAR(10) NOT NULL,
        BookNM VARCHAR(250) NOT NULL,
        AuthorID INT NOT NULL,
        ChapterDesc XML(BookStoreCollection) NULL
       );
GO

: Msg 6328, Level 16, State 1, Line 3
Specified collection 'BookStoreCollection' cannot be dropped because it is used by object 'dbo.BookInfoExport'.

: Msg 6348, Level 16, State 1, Line 5
Specified collection 'BookStoreCollection' cannot be created because it already exists or you do not have permission.

Commands completed successfully.

Total execution time: 00:00:00.031

In [5]:
-- recipe 25-4
SELECT  name
FROM    sys.XML_schema_collections
ORDER BY create_date;

SELECT  n.name
FROM    sys.XML_schema_namespaces n
        INNER JOIN sys.XML_schema_collections c
            ON c.XML_collection_id = n.XML_collection_id
WHERE   c.name = 'BookStoreCollection';
GO

(2 rows affected)

(1 row affected)

Total execution time: 00:00:00.050

name
sys
BookStoreCollection


name
http://PROD/BookStore


In [6]:
-- recipe 25-5
IF OBJECT_ID('dbo.BookInvoice') IS NOT NULL DROP TABLE dbo.BookInvoice;
CREATE  TABLE dbo.BookInvoice
       (
        BookInvoiceID INT IDENTITY PRIMARY  KEY,
        BookInvoiceXML XML NOT  NULL
       )
GO

INSERT  dbo.BookInvoice (BookInvoiceXML)
VALUES  
('<BookInvoice  invoicenumber="1"  customerid="22"     orderdate="2008-07-01Z">
<OrderItems>
<Item  id="22"  qty="1"  name="SQL Fun in the Sun"/>
<Item  id="24"  qty="1"  name="T-SQL Crossword Puzzles"/>
</OrderItems>
</BookInvoice>'),

('<BookInvoice  invoicenumber="1"  customerid="40"  orderdate="2008-07-11Z">
<OrderItems>
<Item  id="11"  qty="1"  name="MCITP Cliff Notes"/>
</OrderItems>
</BookInvoice>'),

('<BookInvoice  invoicenumber="1"  customerid="9"  orderdate="2008-07-22Z">
<OrderItems>
<Item  id="11"  qty="1"  name="MCITP Cliff Notes"/>
<Item  id="24"  qty="1"  name="T-SQL Crossword Puzzles"/>
</OrderItems>
</BookInvoice>');


SELECT  BookInvoiceID
FROM    dbo.BookInvoice
WHERE   BookInvoiceXML.exist('/BookInvoice/OrderItems/Item[@id=11]') = 1;

DECLARE @BookInvoiceXML XML;
SELECT  @BookInvoiceXML = BookInvoiceXML
FROM    dbo.BookInvoice
WHERE   BookInvoiceID = 2;

SELECT  BookID.value('@id', 'integer') BookID
FROM    @BookInvoiceXML.nodes('/BookInvoice/OrderItems/Item') AS BookTable (BookID);
GO

DECLARE @BookInvoiceXML XML;
SELECT  @BookInvoiceXML = BookInvoiceXML
FROM    dbo.BookInvoice
WHERE   BookInvoiceID = 3;
SELECT  @BookInvoiceXML.query('/BookInvoice/OrderItems'); 

SELECT DISTINCT
        BookInvoiceXML.value('(/BookInvoice/OrderItems/Item/@name)[1]',
                             'varchar(30)') AS BookTitles
FROM    dbo.BookInvoice
UNION
SELECT DISTINCT
        BookInvoiceXML.value('(/BookInvoice/OrderItems/Item/@name)[2]',
                             'varchar(30)')
FROM    dbo.BookInvoice;

SELECT DISTINCT
        BookInvoiceXML.value('(/BookInvoice/OrderItems/Item/@name)[3]',
                             'varchar(30)')
FROM    dbo.BookInvoice;


Commands completed successfully.

(3 rows affected)

(2 rows affected)

(1 row affected)

(1 row affected)

...

BookInvoiceID
2
3


BookID
11


(No column name)
"<OrderItems><Item id=""11"" qty=""1"" name=""MCITP Cliff Notes"" /><Item id=""24"" qty=""1"" name=""T-SQL Crossword Puzzles"" /></OrderItems>"


BookTitles
NULL
MCITP Cliff Notes
SQL Fun in the Sun
T-SQL Crossword Puzzles


(No column name)
NULL


In [7]:
-- recipe 25-6
SELECT BookInvoiceXML 
FROM dbo.BookInvoice 
WHERE BookInvoiceID = 2;

UPDATE dbo.BookInvoice
SET BookInvoiceXML.modify
('insert <Item id="920" qty="l" name="SQL Server 2014 Transact-SOL Recipes"/>
into (/BookInvoice/OrderItems)[1]')
WHERE BookInvoiceID = 2;

SELECT BookInvoiceXML 
FROM dbo.BookInvoice 
WHERE BookInvoiceID = 2;


(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.082

BookInvoiceXML
"<BookInvoice invoicenumber=""1"" customerid=""40"" orderdate=""2008-07-11Z""><OrderItems><Item id=""11"" qty=""1"" name=""MCITP Cliff Notes"" /></OrderItems></BookInvoice>"


BookInvoiceXML
"<BookInvoice invoicenumber=""1"" customerid=""40"" orderdate=""2008-07-11Z""><OrderItems><Item id=""11"" qty=""1"" name=""MCITP Cliff Notes"" /><Item id=""920"" qty=""l"" name=""SQL Server 2014 Transact-SOL Recipes"" /></OrderItems></BookInvoice>"


In [8]:
-- recipe 25-7
CREATE PRIMARY XML INDEX idx_XML_Primary_Book_ChapterDESC
ON dbo.Book(ChapterDesc); 

CREATE XML INDEX idx_XML_Value_Book_ChapterDESC ON dbo.Book(ChapterDESC)
USING XML INDEX idx_XML_Primary_Book_ChapterDESC FOR VALUE;


Commands completed successfully.

Total execution time: 00:00:00.057

In [9]:
-- recipe 25-8
SELECT  ShiftID,
        Name
FROM    AdventureWorks2014.HumanResources.[Shift]
FOR     XML RAW('Shift'),
            ROOT('Shifts'),
            TYPE;

SELECT TOP 3
        BusinessEntityID,
        Shift.Name,
        Department.Name
FROM    AdventureWorks2014.HumanResources.EmployeeDepartmentHistory Employee
        INNER JOIN AdventureWorks2014.HumanResources.Shift Shift
            ON Employee.ShiftID = Shift.ShiftID
        INNER JOIN AdventureWorks2014.HumanResources.Department Department
            ON Employee.DepartmentID = Department.DepartmentID
ORDER BY BusinessEntityID
FOR     XML AUTO,
            TYPE;

SELECT TOP 3
        Shift.Name,
        Department.Name,
        BusinessEntityID
FROM    AdventureWorks2014.HumanResources.EmployeeDepartmentHistory Employee
        INNER JOIN AdventureWorks2014.HumanResources.Shift Shift
            ON Employee.ShiftID = Shift.ShiftID
        INNER JOIN AdventureWorks2014.HumanResources.Department Department
            ON Employee.DepartmentID = Department.DepartmentID
ORDER BY Shift.Name,
        Department.Name,
        BusinessEntityID
FOR     XML AUTO,
            TYPE;

SELECT TOP 3
        1 AS Tag,
        NULL AS Parent,
        BusinessEntityID AS [Vendor!1!VendorID],
        Name AS [Vendor!1!VendorName!ELEMENT],
        CreditRating AS [Vendor!1!CreditRating]
FROM    AdventureWorks2014.Purchasing.Vendor
ORDER BY CreditRating
FOR     XML EXPLICIT,
            TYPE;

SELECT  Name AS "@Territory",
        CountryRegionCode AS "@Region",
        SalesYTD
FROM    AdventureWorks2014.Sales.SalesTerritory
WHERE   SalesYTD > 6000000
ORDER BY SalesYTD DESC
FOR     XML PATH('TerritorySales'),
            ROOT('CompanySales'),
            TYPE;

SELECT  Name AS "Territory",
        CountryRegionCode AS "Territory/Region",
        SalesYTD AS "Territory/Region/YTDSales"
FROM    AdventureWorks2014.Sales.SalesTerritory
WHERE   SalesYTD > 6000000
ORDER BY SalesYTD DESC
FOR     XML PATH('TerritorySales'),
            ROOT('CompanySales'),
            TYPE;

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(No column name)
"<Employee BusinessEntityID=""1""><Shift Name=""Day""><Department Name=""Executive"" /></Shift></Employee><Employee BusinessEntityID=""2""><Shift Name=""Day""><Department Name=""Engineering"" /></Shift></Employee><Employee BusinessEntityID=""3""><Shift Name=""Day""><Department Name=""Engineering"" /></Shift></Employee>"


(No column name)
"<Shift Name=""Day""><Department Name=""Document Control""><Employee BusinessEntityID=""217"" /><Employee BusinessEntityID=""219"" /><Employee BusinessEntityID=""220"" /></Department></Shift>"


(No column name)
"<Vendor VendorID=""1496"" CreditRating=""1""><VendorName>Advanced Bicycles</VendorName></Vendor><Vendor VendorID=""1492"" CreditRating=""1""><VendorName>Australia Bike Retailer</VendorName></Vendor><Vendor VendorID=""1500"" CreditRating=""1""><VendorName>Morgan Bike Accessories</VendorName></Vendor>"


(No column name)
"<CompanySales><TerritorySales Territory=""Southwest"" Region=""US""><SalesYTD>10510853.8739</SalesYTD></TerritorySales><TerritorySales Territory=""Northwest"" Region=""US""><SalesYTD>7887186.7882</SalesYTD></TerritorySales><TerritorySales Territory=""Canada"" Region=""CA""><SalesYTD>6771829.1376</SalesYTD></TerritorySales></CompanySales>"


(No column name)
<CompanySales><TerritorySales><Territory>Southwest<Region>US<YTDSales>10510853.8739</YTDSales></Region></Territory></TerritorySales><TerritorySales><Territory>Northwest<Region>US<YTDSales>7887186.7882</YTDSales></Region></Territory></TerritorySales><TerritorySales><Territory>Canada<Region>CA<YTDSales>6771829.1376</YTDSales></Region></Territory></TerritorySales></CompanySales>


In [10]:
-- recipe 25-9
DECLARE @XMLdoc XML,
        @iDoc   INTEGER;
SET  @XMLdoc  =
'<Book  name="SQL Server 2000 Fast Answers">
    <Chapters>
        <Chapter  id="1"  name="Installation, Upgrades"/>
        <Chapter  id="2"  name="Configuring SQL Server"/>
        <Chapter  id="3"  name="Creating and Configuring Databases"/>
        <Chapter  id="4"  name="SQL Server Agent and SQL Logs"/>
    </Chapters>
</Book>';

EXECUTE sp_XML_preparedocument @iDoc OUTPUT, @XMLdoc;

SELECT Chapter, ChapterNm
FROM OPENXML(@iDoc, '/Book/Chapters/Chapter', 0)
WITH (Chapter INT '@id', ChapterNm VARCHAR(50) '@name');

EXECUTE sp_xml_removedocument @idoc;

(4 rows affected)

Total execution time: 00:00:00.135

Chapter,ChapterNm
1,"Installation, Upgrades"
2,Configuring SQL Server
3,Creating and Configuring Databases
4,SQL Server Agent and SQL Logs


In [11]:
-- recipe 25-10
SELECT STUFF((SELECT TOP (25) ',' + CONVERT(VARCHAR(15), BusinessEntityID)
              FROM AdventureWorks2014.HumanResources.Employee
              ORDER BY BusinessEntityID
              FOR XML PATH(''), TYPE).value('.','varchar(max)'),1,1,'');

-- cleanup
IF OBJECT_ID('dbo.Book') IS NOT NULL DROP TABLE dbo.Book;
IF OBJECT_ID('dbo.INS_Book','P') IS NOT NULL DROP PROCEDURE dbo.INS_Book;
IF OBJECT_ID('dbo.BookInfoExport') IS NOT NULL DROP TABLE dbo.BookInfoExport;
IF OBJECT_ID('dbo.BookInvoice') IS NOT NULL DROP TABLE dbo.BookInvoice;

IF EXISTS (SELECT * FROM sys.xml_schema_collections WHERE name = 'BookStoreCollection')
DROP XML SCHEMA COLLECTION BookStoreCollection;
-- 

(1 row affected)

Total execution time: 00:00:00.062

(No column name)
"1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25"
